In [3]:
#IMPORTS

import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import matplotlib
import os

print("Librosa version:", librosa.__version__)
print("Numpy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Python version:", np.__version__)

Librosa version: 0.11.0
Numpy version: 2.0.2
Matplotlib version: 3.10.0
Python version: 2.0.2


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#NAHRANI SOUBORU Z PC

from google.colab import files
uploaded = files.upload()

import shutil
import os

# Cílová složka na Google Drive
target_folder = "/content/drive/MyDrive/gunshots_wav"
os.makedirs(target_folder, exist_ok=True)

# Přesun každého souboru
for fname in uploaded.keys():
    shutil.move(fname, os.path.join(target_folder, fname))

print("✅ Vše nahráno do složky:", target_folder)

✅ Vše nahráno do složky: /content/drive/MyDrive/gunshots_wav


In [4]:
def get_spectrogram(path, sr=22050, n_mels=64, hop_length=1024, n_fft=2048):
    y, sr = librosa.load(path, sr=sr, mono=True, duration=2.0)  # limit 2 s
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft,
                                       hop_length=hop_length, n_mels=n_mels)
    return librosa.power_to_db(S, ref=np.max)


In [5]:
def extract_mfcc(path, n_mfcc=13, duration=4.0, sr=22050):
    y, _ = librosa.load(path, sr=sr, duration=duration)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)  # průměr přes časové okno

In [9]:
import os, numpy as np, gc
import soundata

# Inicializace UrbanSound8K
dataset = soundata.initialize("urbansound8k")
print("📦 Kontroluji dataset...")
dataset.download()
dataset.validate()
clip_ids = dataset.clip_ids
print(f"🔍 V databázi je {len(clip_ids)} klipů.")

# Kategorie
target_labels = ['gun_shot']
negative_labels = ['children_playing', 'air_conditioner', 'dog_bark']

# (volitelně) omezení vzorků kvůli RAM – nastav číslo nebo nech None pro celé
MAX_POS = None   # např. 1500
MAX_NEG = None   # např. 1500

X, y = [], []
pos = neg = 0

# 🔊 1) UrbanSound8K
for clip_id in clip_ids:
    clip = dataset.clip(clip_id)
    if clip.tags and clip.tags.labels:
        label = clip.tags.labels[0]
        if label in target_labels + negative_labels:
            # limity pro RAM
            if label == 'gun_shot' and MAX_POS is not None and pos >= MAX_POS:
                continue
            if label != 'gun_shot' and MAX_NEG is not None and neg >= MAX_NEG:
                continue
            try:
                spec = get_spectrogram(clip.audio_path)  # definuješ výš
                X.append(spec)
                if label == 'gun_shot':
                    y.append(1); pos += 1
                else:
                    y.append(0); neg += 1
            except Exception as e:
                print(f"❌ Chyba u UrbanSound {clip.audio_path}: {e}")

# 🔫 2) Vlastní WAV z Drive
FREESOUND_WAV_DIR = "/content/drive/MyDrive/gunshots_wav"
if os.path.isdir(FREESOUND_WAV_DIR):
    for filename in os.listdir(FREESOUND_WAV_DIR):
        if filename.lower().endswith(".wav"):
            try:
                path = os.path.join(FREESOUND_WAV_DIR, filename)
                spec = get_spectrogram(path)
                X.append(spec)
                y.append(1)  # tvoje freesoundy jsou gunshot
            except Exception as e:
                print(f"❌ Chyba u Freesound {filename}: {e}")
else:
    print(f"ℹ️ Složka {FREESOUND_WAV_DIR} nenalezena – přeskočeno.")

# ⚖️ sjednocení časové délky (pad/trim)
max_time = max(s.shape[1] for s in X)
X_pad = []
for spec in X:
    t = spec.shape[1]
    if t < max_time:
        spec = np.pad(spec, ((0,0),(0, max_time - t)), mode='constant')
    elif t > max_time:
        spec = spec[:, :max_time]
    X_pad.append(spec)

# ➜ převod na NumPy + úsporné typy až teď
X = np.array(X_pad, dtype=np.float16)[..., np.newaxis]  # (N, mels, time, 1)
y = np.array(y, dtype=np.int8)
del X_pad; gc.collect()

print("✅ Dataset shape:", X.shape, y.shape, "| dtypes:", X.dtype, y.dtype,
      f"| ~{X.nbytes/1024/1024:.1f} MiB pro X")

📦 Kontroluji dataset...


 50%|█████     | 2.82G/5.61G [26:32<26:15, 1.90MB/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3385728372.py", line 7, in <cell line: 0>
    dataset.download()
  File "/usr/local/lib/python3.12/dist-packages/soundata/core.py", line 328, in download
    download_utils.downloader(
  File "/usr/local/lib/python3.12/dist-packages/soundata/download_utils.py", line 141, in downloader
    download_tar_file(remotes[k], save_dir, force_overwrite, cleanup)
  File "/usr/local/lib/python3.12/dist-packages/soundata/download_utils.py", line 386, in download_tar_file
    tar_download_path = download_from_remote(tar_remote, save_dir, force_overwrite)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/soundata/download_utils.py", line 252, in download_from_remote
    urllib.request.urlretrieve(
  F

TypeError: object of type 'NoneType' has no len()

In [7]:
# Split na tréninkovou a testovací sadu
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Model Neurounky

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Resizing, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D

input_shape = X_train.shape[1:]

model = Sequential([
    Resizing(64, 64, input_shape=input_shape),
    Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.save("gunshot_cnn_model.h5")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 62, 62, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,977 (27.25 KB)

 Trainable params: 6,977 (27.25 KB)

 Non-trainable params: 0 (0.00 B)

In [2]:
from tensorflow.keras.models import load_model

# Načti uložený model
model = load_model("gunshot_cnn_model.h5")

# Ověř si, že je načtený
model.summary()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'gunshot_cnn_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [9]:
# Trénink modelu

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=4)

Epoch 1/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.7814 - loss: 0.4721 - val_accuracy: 0.8965 - val_loss: 0.2859
Epoch 2/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.8850 - loss: 0.3112 - val_accuracy: 0.9083 - val_loss: 0.2319
Epoch 3/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.9098 - loss: 0.2514 - val_accuracy: 0.9228 - val_loss: 0.1922
Epoch 4/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.9098 - loss: 0.2325 - val_accuracy: 0.9373 - val_loss: 0.1676
Epoch 5/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.9309 - loss: 0.1885 - val_accuracy: 0.9437 - val_loss: 0.1521
Epoch 6/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.9279 - loss: 0.1866 - val_accuracy: 0.9446 - val_loss: 0.1582
Epoch 7/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.9387 - loss: 0.1643 - val_accuracy: 0.9573 - val_loss: 0.1357
Epoch 8/10
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.9435 -

In [10]:
# Vyhodnocení modelu

loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test accuracy: {acc:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9540 - loss: 0.1354
✅ Test accuracy: 0.9482


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

# Předpokládám, že máš y_test (skutečné hodnoty) a y_pred (predikce modelu)
# Pokud je to klasifikační úloha s pravděpodobnostmi, tak ještě y_prob = model.predict_proba(X_test)

# ✅ Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.show()

# ✅ ROC křivka (binary klasifikace)
y_prob = model.predict(X_test)  # pokud máš pravděpodobnosti (např. sigmoid výstup)
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC)")
plt.legend(loc="lower right")
plt.show()


NameError: name 'y_test' is not defined